
# **Parte C — Modelagem & Resultados**  
**Dataset:** Heart Failure Prediction (918 instâncias combinadas das bases UCI)  
**Modelos:** Regressão Logística e k-NN  
**Autores:** _[preencha com os 3 nomes]_

**Objetivo:** Comparar o desempenho de dois algoritmos clássicos (Regressão Logística e k-NN) para o problema de classificação binária de doença cardíaca, utilizando os dados **já pré-processados** (normalização, codificação e split treino/teste prontos).


In [ ]:

# =========================
# Imports
# =========================
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             classification_report, confusion_matrix, roc_curve, auc)

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')



## 1. Carregamento dos dados
Carregamos os **arquivos pré-processados** gerados na Parte B:
- `X_train.csv`, `X_test.csv`: atributos (features)
- `y_train.csv`, `y_test.csv`: alvo (binário: 0 = sem doença, 1 = com doença)


In [ ]:

# =========================
# Load data (assume CSVs in same directory as notebook)
# =========================
X_train = pd.read_csv('X_train.csv')
X_test  = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv').iloc[:, 0]
y_test  = pd.read_csv('y_test.csv').iloc[:, 0]

print("X_train:", X_train.shape, "| X_test:", X_test.shape)
print("y_train:", y_train.shape, "| y_test:", y_test.shape)

print("\nProporção de classes (treino):")
print(y_train.value_counts(normalize=True).round(3))



## 2. Funções auxiliares de avaliação
Funções para avaliar o modelo e gerar métricas e gráficos (matriz de confusão e curva ROC).


In [ ]:

def evaluate_model(model, X_tr, y_tr, X_te, y_te, model_name="Modelo"):
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_te)[:, 1]
    else:
        try:
            y_dec = model.decision_function(X_te)
            y_prob = (y_dec - y_dec.min()) / (y_dec.max() - y_dec.min() + 1e-12)
        except Exception:
            y_prob = None

    metrics = {
        "Modelo": model_name,
        "Acuracia": accuracy_score(y_te, y_pred),
        "Precisao": precision_score(y_te, y_pred, zero_division=0),
        "Recall": recall_score(y_te, y_pred, zero_division=0),
        "F1": f1_score(y_te, y_pred, zero_division=0)
    }
    if y_prob is not None:
        fpr, tpr, _ = roc_curve(y_te, y_prob)
        roc_auc = auc(fpr, tpr)
        metrics["AUC"] = roc_auc
    else:
        fpr, tpr, roc_auc = None, None, None

    report = classification_report(y_te, y_pred, zero_division=0)
    cm = confusion_matrix(y_te, y_pred)

    return metrics, report, cm, (fpr, tpr, roc_auc), y_pred, y_prob


def plot_confusion_matrix(cm, title="Matriz de Confusão"):
    fig, ax = plt.subplots()
    cax = ax.imshow(cm, interpolation='nearest')
    ax.set_title(title)
    fig.colorbar(cax)
    tick_marks = np.arange(2)
    ax.set_xticks(tick_marks)
    ax.set_yticks(tick_marks)
    ax.set_xticklabels(['Sem doença (0)', 'Com doença (1)'])
    ax.set_yticklabels(['Sem doença (0)', 'Com doença (1)'])
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    ax.set_ylabel('Verdadeiro')
    ax.set_xlabel('Predito')
    plt.show()


def plot_roc_curve(fpr, tpr, roc_auc, title="Curva ROC"):
    if fpr is None or tpr is None:
        print("Este modelo não fornece probabilidades para plotar ROC.")
        return
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.title(title)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    plt.show()



## 3. Validação cruzada (k-fold) e busca de hiperparâmetros
Usamos `StratifiedKFold` (k=5) para manter a proporção de classes em cada *fold*.


In [ ]:

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)



## 4. Regressão Logística (baseline probabilístico)
Fazemos `GridSearchCV` simples sobre **C** (força da regularização) e escolhemos `liblinear` para garantir convergência rápida.


In [ ]:

log_reg = LogisticRegression(max_iter=1000, solver='liblinear', random_state=42)

param_grid_lr = {
    'C': [0.01, 0.1, 1.0, 10.0, 100.0],
    'penalty': ['l2']
}

grid_lr = GridSearchCV(log_reg, param_grid_lr, cv=cv, scoring='f1', n_jobs=-1)
grid_lr.fit(X_train, y_train)

best_lr = grid_lr.best_estimator_
print("Melhor Regressão Logística:", grid_lr.best_params_)
print("F1 médio (CV):", grid_lr.best_score_)


In [ ]:

metrics_lr, report_lr, cm_lr, roc_lr, ypred_lr, yprob_lr = evaluate_model(best_lr, X_train, y_train, X_test, y_test, model_name="LogisticRegression")

print("== Regressão Logística: Métricas no Teste ==")
print(pd.Series(metrics_lr))

print("\nRelatório de Classificação:")
print(report_lr)

plot_confusion_matrix(cm_lr, title="Matriz de Confusão — Regressão Logística")
plot_roc_curve(*roc_lr, title="Curva ROC — Regressão Logística")



## 5. k-Nearest Neighbors (k-NN)
Como os dados foram **normalizados** na Parte B, o k-NN é adequado. Fazemos grid em `k` (n_neighbors), métrica e pesos.


In [ ]:

knn = KNeighborsClassifier()

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(knn, param_grid_knn, cv=cv, scoring='f1', n_jobs=-1)
grid_knn.fit(X_train, y_train)

best_knn = grid_knn.best_estimator_
print("Melhor KNN:", grid_knn.best_params_)
print("F1 médio (CV):", grid_knn.best_score_)


In [ ]:

metrics_knn, report_knn, cm_knn, roc_knn, ypred_knn, yprob_knn = evaluate_model(best_knn, X_train, y_train, X_test, y_test, model_name="KNN")

print("== KNN: Métricas no Teste ==")
print(pd.Series(metrics_knn))

print("\nRelatório de Classificação:")
print(report_knn)

plot_confusion_matrix(cm_knn, title="Matriz de Confusão — KNN")
plot_roc_curve(*roc_knn, title="Curva ROC — KNN")



## 6. Comparação final
Tabela agregando as principais métricas de cada modelo.


In [ ]:

from caas_jupyter_tools import display_dataframe_to_user

df_results = pd.DataFrame([metrics_lr, metrics_knn])
cols = ['Modelo', 'Acuracia', 'Precisao', 'Recall', 'F1']
if 'AUC' in df_results.columns:
    cols.append('AUC')
df_results = df_results[cols]

print(df_results)
display_dataframe_to_user("Resultados - LogReg vs KNN", df_results)



## 7. Observações rápidas (para o relatório)
- **Regressão Logística**: baseline probabilístico, interpretável; bom equilíbrio entre **precisão** e **recall**.  
- **k-NN**: sensível à escolha de *k* e à métrica; pode ter recall inferior se houver ruído.  
- **Matriz de Confusão**: analise os **falsos negativos** (pacientes com doença predita como saudáveis) — isso é crítico em aplicações clínicas.  
- **AUC/ROC**: útil para comparar a capacidade de separação entre as classes.
